In [1]:
import os 
import cv2
import requests
import numpy as np
import random
import json
import math 
import albumentations as A
from pprint import pprint

/home/hbdesk/labelstudio_convert/venv/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
LABEL_STUDIO_KEY="eaffb8f4719efb17f9b227ed56fee991c4d06a0a"
LABEL_STUDIO_URL="http://192.168.231.52:8080/"

def bgr2_3grey(image: np.ndarray):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_bgr = cv2.merge([gray_image, gray_image, gray_image])
    return gray_bgr
    
def get_img_from_studio(image_path, save_path="./captioned_images"):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    download_url = f"{LABEL_STUDIO_URL}{image_path}"
    headers = {
        'Authorization': f'Token {LABEL_STUDIO_KEY}'
    }
    response = requests.get(download_url, headers=headers)
    if response.status_code == 200:
        filename = os.path.basename(image_path)
        save_path = os.path.join(save_path, filename)
        with open(save_path, 'wb') as f:
            f.write(response.content)
            return save_path
        print(f"Image successfully downloaded as {filename}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")
        return None

def export_label_txt(label_dict_list: list, txt_type: str):
    """
    takes a list of annotation pairs that looks like 
    `Image_path`\t`Label`\n
    """
    if txt_type.lower() == "train":
        with open("rec_gt_train.txt", "w", encoding='utf-8') as train_txt:
            for _a in label_dict_list:
                for key, value in _a.items():
                    train_txt.write(f"{str(key)}\t{str(value)}\n")
    elif txt_type.lower() == "eval":
        with open("rec_gt_eval.txt", "w", encoding='utf-8') as eval_txt:
            for _a in label_dict_list:
                for key, value in _a.items():
                    eval_txt.write(f"{key}\t{str(value)}\n")
    else:
        print(f"invalid export type {txt_type}!")

def separate_train_eval(all_anno: list,train_sep: float = 0.8):
    _all_anno = all_anno.copy() # copy so we dont fuck up the original shit homie
    _train_num = math.floor(len(_all_anno) * train_sep) 
    train_set = []
    _sel_count = _train_num
    for _ in range(_train_num):
        random_index = random.randint(0, (_sel_count - 1))
        _sel_count -= 1
        _sel  = _all_anno[random_index]
        train_set.append(_sel)
        # remove from the copied array and use remaining as eval, avoid copying twice
        _all_anno.pop(_all_anno.index(_sel)) 
    return train_set, _all_anno 

def convert2gray(file_path: str):
    for file in os.listdir(file_path):
        img_path = os.path.join(file_path, file)
        img = bgr2_3grey(cv2.imread(img_path))
        cv2.imwrite(img_path, img)

In [3]:
def new_query_string(img_path: str, img_caption: str):
    return {
              "query": "<image>",
              "response": f"{img_caption}",
              "images": [img_path]
            }

In [14]:
with open("./2.7_BOLD.json", "r") as load_labelled:
    data = json.loads(load_labelled.read())
pprint(data[0])

{'annotations': [{'completed_by': 2,
                  'created_at': '2024-11-01T07:06:27.712531Z',
                  'draft_created_at': None,
                  'ground_truth': False,
                  'id': 2088,
                  'import_id': None,
                  'last_action': None,
                  'last_created_by': None,
                  'lead_time': 17.095,
                  'parent_annotation': None,
                  'parent_prediction': None,
                  'prediction': {},
                  'project': 6,
                  'result': [{'from_name': 'caption',
                              'id': '9U2QMRVai8',
                              'origin': 'manual',
                              'to_name': 'image',
                              'type': 'textarea',
                              'value': {'text': ['PeRo_longdd']}}],
                  'result_count': 0,
                  'task': 2852,
                  'unique_id': '3d9f9cc4-bd8e-4030-887c-dcbfa7680d11',
       

In [15]:
final_query_list = []
for annotation in data:
    try:
        text_string = annotation["annotations"][0]["result"][0]["value"]["text"][0]
        file_name = annotation["data"]["captioning"]
        image_name = get_img_from_studio(file_name, "/home/hbdesk/labelstudio_convert/captioned_images")
        if image_name is not None:
            query_ = new_query_string(image_name, text_string)
            final_query_list.append(query_)
    except Exception as _:
        continue

In [16]:
with open("GOT2_data.jsonl", "w") as writejson:
    for item in final_query_list:
        writejson.write(json.dumps(item) + "\n")